In [1]:
# necessary libraries for prediciton
import pandas as pd
import numpy as np
import os

In [13]:
def read_csv_to_df(filename):
    '''Read a csv to a pandas dataframe'''
    return pd.read_csv(filename)

In [24]:
def write_df_to_csv(df, directory, filename):
    '''Write the content of a dataframe to a csv file'''
    csv = df.to_csv(index=False)

    os.makedirs(directory, exist_ok=True)
    output_path = '%s/%s' % (directory, filename)

    with open(output_path, 'w') as fd:
        fd.write(csv)

# Predictions

### For a first simpler approach, we will only use the 'loan' table

In [9]:
# Reading the loan train & loan test table
loan_df = read_csv_to_df('competition_dataset/loan_train.csv')

In [10]:
# We should now start 'cleansing' the data here

In [11]:
# Good tutorial for feature engineering:
# https://medium.com/datadriveninvestor/a-simple-guide-to-creating-predictive-models-in-python-part-1-8e3ddc3d7008

In [1]:
# Outputting the resultant table to a final csv
write_df_to_csv(loan_df, 'dataset', 'preprocessed_data.csv')

NameError: name 'write_df_to_csv' is not defined